In [155]:
import pandas as pd
import numpy as np

#Dados importados do perfil Vini0804 em 14/04/2024

diary_df = pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\diary.csv")
likes_df = pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\likes\films.csv")
ratings_df= pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\ratings.csv")
watched_df=pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\watched.csv")

# Manipulando os dados para juntar todos em um só dataframe 

watched_df =  watched_df.drop('Date', axis=1)  # Removendo a coluna Date pois ela não sera usada
watched_df["Like"]=watched_df["Name"].isin(likes_df["Name"]) # Criando a  coluna Like com base nos filmes que estao no dataframe Likes


#print(watched_df)

In [156]:
# Adicionando a data de quando o filme foi visto 

watched_df['First Log Date'] = np.nan  # Criando uma nova coluna 

for indice_watch, filme in enumerate(watched_df["Name"]):
    for indice_diario, filme_diario in enumerate(diary_df["Name"]):
        if filme == filme_diario:
            watched_df.loc[indice_watch, "First Log Date"] = diary_df.loc[indice_diario, "Watched Date"]
            
#print(watched_df)

In [157]:
# Adicionando o rating do filme 

watched_df['Rating'] = np.nan  # Criando uma nova coluna 

for indice_watch, filme in enumerate(watched_df["Name"]):
    for indice_rating, filme_rating in enumerate(ratings_df["Name"]):
        if filme == filme_rating:
            watched_df.loc[indice_watch, "Rating"] = ratings_df.loc[indice_rating, "Rating"]
            
#print(watched_df)

In [158]:
# Contador de filmes reassistidos

watched_df['Rewatch'] = 0
for indice_watch, filme in enumerate(watched_df["Name"]):
    for indice_diario, filme_diario in enumerate(diary_df["Name"]):
        if filme == filme_diario and diary_df["Rewatch"][indice_diario]=="Yes":
            watched_df.loc[indice_watch, 'Rewatch'] += 1

print(watched_df)


                                    Name  Year        Letterboxd URI   Like  \
0                         Captain Marvel  2019  https://boxd.it/9vSA  False   
1      Spider-Man: Into the Spider-Verse  2018  https://boxd.it/azpY   True   
2                          The Favourite  2018  https://boxd.it/d2hI  False   
3                             Green Book  2018  https://boxd.it/hErk  False   
4                         A Star Is Born  2018  https://boxd.it/aXgc  False   
...                                  ...   ...                   ...    ...   
1598  Before the Devil Knows You're Dead  2007  https://boxd.it/20Jy   True   
1599                  Orion and the Dark  2024  https://boxd.it/H2TW  False   
1600                              Priest  2011  https://boxd.it/16fM  False   
1601                        Philadelphia  1993  https://boxd.it/1XMk  False   
1602                Cheaper by the Dozen  2003  https://boxd.it/1USa  False   

     First Log Date  Rating  Rewatch  
0           